In [1]:
import pandas as pd

In [ ]:
import urllib
import xmltodict

In [131]:
#samples = pd.read_csv( 'tumor_details.csv' )

In [132]:
#samples.head()

In [19]:
def do_request( url ):
    file = urllib.request.urlopen( url )
    data = file.read()
    file.close()
    data = xmltodict.parse(data)
    return data

In [20]:
def get_all_samples():
    return do_request( 'http://api.brain-map.org/api/v2/data/query.xml?criteria=model::Specimen,rma::criteria,products[name$eq%27Glioblastoma%27],rma::options[num_rows$eqall]' )

In [130]:
all_samples_resp = get_all_samples()

In [22]:
all_specimens = sorted( [ x['external-specimen-name'] for x in all_samples_resp['Response']['specimens']['specimen'] ] )
print( 'Found {} samples'.format( len(all_specimens) ) )

Found 948 samples


In [82]:
def get_sample_detail( specimen='W1-1-2-A.1.01' ):
    return do_request( "http://api.brain-map.org/api/v2/data/query.xml?criteria=model::SectionDataSet,rma::criteria,specimen[external_specimen_name$eq'{}'],rma::include,genes,sub_images".format(specimen) )

In [172]:
def get_all_section_images( specimen='W1-1-2-A.1.01' ):
    data = get_sample_detail( specimen )
    data = data['Response']['section-data-sets']['section-data-set']
    
    out = []
    for a in data:
        if a['genes'] is not None:
            out.append(
                (
                    a['genes']['gene']['acronym'],
                    a['sub-images']['sub-image']['id']['#text'],
                    a['sub-images']['sub-image']['section-number']['#text']
                )
            )
        else:
            for i,sub_image in enumerate( a['sub-images']['sub-image'] ):
                try:
                    out.append( ('H+E', sub_image['id']['#text'], sub_image['section-number']['#text']) )
                except:
                    pass
                
    for i,a in enumerate(out):
        a = out[i]
        out[i] = ( a[0], int(a[1]), int(a[2]) )
        
    return sorted( out, key=lambda x: x[2] )

In [170]:
get_all_section_images( specimen='W1-1-2-A.1.02' )

[('H+E', 292324585, 2),
 ('TGFBR2', 287502780, 3),
 ('BIRC5', 287502729, 4),
 ('H+E', 292324567, 5),
 ('TNFAIP3', 287502677, 6),
 ('HIF1A', 287502626, 7),
 ('H+E', 292324549, 8),
 ('ITGA6', 287502574, 9),
 ('OLIG2', 287502523, 10),
 ('H+E', 292324519, 11),
 ('ID1', 287502472, 12),
 ('PDGFRA', 287502419, 13),
 ('H+E', 292324711, 14),
 ('POSTN', 287502368, 15),
 ('PROM1', 287502317, 16),
 ('H+E', 292324693, 17),
 ('EZH2', 287502266, 18),
 ('DANCR', 287502215, 19),
 ('H+E', 292324675, 20),
 ('NOS2', 287502164, 21),
 ('CD44', 287502113, 22),
 ('H+E', 292324657, 23),
 ('IGFBP2', 287502061, 24),
 ('MET', 287502010, 25),
 ('H+E', 292324639, 26),
 ('MYC', 287501958, 27),
 ('ID2', 287501907, 28),
 ('H+E', 292324621, 29),
 ('PDPN', 287501856, 30),
 ('PI3', 287501805, 31),
 ('H+E', 292324603, 32)]

In [67]:
def download_image_from_id( id_str, name=None, feature_map=False ):
    if not name:
        name = id_str
        
    url = "http://api.brain-map.org/api/v2/image_download/{}".format(id_str)
    if feature_map:
        url += '?view=tumor_feature_annotation'
        
    urllib.request.urlretrieve(url, "{}.jpg".format(name))

In [129]:
download_image_from_id( '292324238',feature_map=True )

In [176]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

[1, 4, 9]
